In [4]:
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.manifold import SpectralEmbedding
import sys
sys.path.append("..")
# print(os.getcwd())

In [8]:
df = pd.read_csv("../data/PEMS08/distance.csv")
assert set(df.columns) == {"from", "to", "cost"}


num_nodes = 170

# ============================================================
# 2. 构建无向图（加权图：权重为距离）
# ============================================================
G = nx.Graph()

# 添加节点
G.add_nodes_from(range(num_nodes))

# 添加边（距离越小表示越接近）
for _, row in df.iterrows():
    u = int(row["from"])
    v = int(row["to"])
    w = float(row["cost"])
    G.add_edge(u, v, weight=w)

# ============================================================
# 3. 生成图的加权邻接矩阵
# ============================================================
A = nx.to_numpy_array(G, nodelist=range(num_nodes), weight="weight")

# 一般 Laplacian Eigenmaps 用 "相似度矩阵"，距离越远相似度越小
# 所以可以进行距离转相似度，例如 Gaussian kernel

sigma = np.mean(A[A > 0])  # 平均距离作为尺度
similarity = np.exp(-A**2 / (2 * sigma**2))
np.fill_diagonal(similarity, 1.0)

# ============================================================
# 4. 计算图拉普拉斯嵌入（Laplacian Eigenmaps / Spectral Embedding）
#    n_components 为嵌入维度，可根据需要调整
# ============================================================
embedding_dim = 16

embedder = SpectralEmbedding(
    n_components=embedding_dim,
    affinity='precomputed'
)

node_embeddings = embedder.fit_transform(similarity)

print("节点嵌入 shape:", node_embeddings.shape)
print("节点 0 的嵌入向量:", node_embeddings[0])

节点嵌入 shape: (170, 16)
节点 0 的嵌入向量: [ 5.43023599e-06  3.66970221e-04 -9.65277405e-05  4.98511434e-05
 -9.75737912e-06  1.04673725e-04  1.02912699e-04 -8.77582491e-05
  2.04718166e-04  8.77234844e-05 -1.96817810e-04  7.67453166e-05
  3.30972233e-06 -7.28170246e-05  1.93232117e-04 -3.20681087e-05]
